## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-28-18-55-58 +0000,nypost,Nvidia to build AI supercomputers for US Enegr...,https://nypost.com/2025/10/28/business/nvidia-...
1,2025-10-28-18-53-50 +0000,nyt,Father-Son Bonding at 2 Miles Per Minute on Ge...,https://www.nytimes.com/2025/10/28/world/europ...
2,2025-10-28-18-53-45 +0000,nyt,Johnson Dismisses Trump 2028 Talk — But Applau...,https://www.nytimes.com/2025/10/28/us/politics...
3,2025-10-28-18-51-28 +0000,nyt,What Zohran Mamdani Learned as an Africana Stu...,https://www.nytimes.com/2025/10/28/us/politics...
4,2025-10-28-18-49-43 +0000,bbc,Brigitte Macron's life 'deteriorated' after al...,https://www.bbc.com/news/articles/czr16vjk8zlo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2365/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,46
253,new,23
42,hurricane,18
43,melissa,17
123,shutdown,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
165,2025-10-28-12-01-13 +0000,wapo,Japan’s new leader tries to charm Trump with g...,https://www.washingtonpost.com/politics/2025/1...,108
229,2025-10-28-07-06-06 +0000,bbc,China controls the rare earths the world buys ...,https://www.bbc.com/news/articles/cj6ny24j0r3o...,100
168,2025-10-28-11-49-55 +0000,nypost,Toyota pledges $10B toward new auto plants in ...,https://nypost.com/2025/10/28/us-news/trump-to...,98
79,2025-10-28-16-33-53 +0000,nypost,Mark Zuckerberg secretly met with AG Pam Bondi...,https://nypost.com/2025/10/28/business/mark-zu...,90
14,2025-10-28-18-40-40 +0000,nyt,Dozens of States Sue Trump Administration Over...,https://www.nytimes.com/2025/10/28/us/politics...,89


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
165,108,2025-10-28-12-01-13 +0000,wapo,Japan’s new leader tries to charm Trump with g...,https://www.washingtonpost.com/politics/2025/1...
299,84,2025-10-27-21-54-59 +0000,nypost,Storm chaser captures stunning eye of Hurrican...,https://nypost.com/2025/10/27/world-news/hurri...
91,58,2025-10-28-16-09-00 +0000,wsj,The Republican-led House Oversight Committee i...,https://www.wsj.com/politics/policy/republican...
30,53,2025-10-28-18-19-57 +0000,nypost,Experts fear betting markets could influence N...,https://nypost.com/2025/10/28/business/pro-mam...
92,50,2025-10-28-16-00-14 +0000,nypost,"NY, 25 other states sue USDA to keep SNAP bene...",https://nypost.com/2025/10/28/us-news/26-state...
41,47,2025-10-28-17-53-26 +0000,wapo,U.S. military strikes four alleged drug boats ...,https://www.washingtonpost.com/national-securi...
102,45,2025-10-28-15-26-21 +0000,nypost,Bill Gates makes major climate change reversal...,https://nypost.com/2025/10/28/us-news/bill-gat...
257,34,2025-10-28-01-00-00 +0000,wsj,Watering down dish soap. Trading down to groce...,https://www.wsj.com/economy/consumers/the-leng...
246,33,2025-10-28-02-19-38 +0000,bbc,Brother of journalist killed in occupied West ...,https://www.bbc.com/news/articles/cx2dnz5zvgxo...
274,32,2025-10-27-23-52-00 +0000,wsj,"“Both political parties have made their point,...",https://www.wsj.com/politics/policy/union-pres...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
